In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important }</style>"))

In [2]:
from utilities import *

import joblib
import tensorflow as tf
from sklearn.preprocessing import normalize

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [4]:
def linear_impute(data):
    for s in scanners:
        dx = coor[0] - ble[s]['X']
        dy = coor[1] - ble[s]['Y']
        d = np.sqrt(np.square(dy) + np.square(dx))
        reg = joblib.load('../Models/linear_distance_models_2/{}.joblib'.format(s))
        ss = pd.Series(reg.predict(np.repeat(d, data.shape[0]).reshape((-1, 1))))
        ss.index = data.index
        data[s].fillna(ss, inplace=True)

def impute(data):
    rolling_win = 30

    data_rolled = data.rolling(rolling_win, min_periods=1).mean()
    data_imputed = data_rolled.ffill()
    linear_impute(data_imputed)
    return data_imputed

In [5]:
ble = read_ble('../Data/ble.csv')

In [6]:
coor = np.array([25, 9])

In [7]:
model = tf.keras.models.load_model("../Models/MLP_CombineData.h5")

In [8]:
client = InfluxDBClient()
data = client.retrieveData(seconds=5, beacon="0117C55D14E4")

In [9]:
data = format_data(data)

In [10]:
data[scanners] = impute(data[scanners])
data[scanners] = normalize(data[scanners] + 100)

In [11]:
data

,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
2020-01-16 03:37:32.525000+00:00,0.172469,0.286483,0.238976,0.203200,0.249011,0.350904,0.312810,0.145494,0.263365,0.087412,0.289559,0.257486,0.259764,0.073509,0.323088,0.235726,0.160140
2020-01-16 03:37:32.557000+00:00,0.171649,0.285120,0.237839,0.202233,0.247826,0.362570,0.311322,0.144802,0.262112,0.086997,0.288181,0.256261,0.258529,0.073159,0.321551,0.234604,0.159378
2020-01-16 03:37:32.565000+00:00,0.171256,0.284468,0.237295,0.212789,0.247260,0.361741,0.310610,0.144471,0.261512,0.086798,0.287522,0.255675,0.257938,0.072992,0.320816,0.234068,0.159014
2020-01-16 03:37:32.598000+00:00,0.168308,0.279570,0.233210,0.209125,0.243002,0.355513,0.305262,0.141984,0.257010,0.085303,0.282572,0.251273,0.313688,0.071735,0.315292,0.230038,0.156276
2020-01-16 03:37:32.605000+00:00,0.172197,0.286031,0.238598,0.213957,0.248617,0.363728,0.312316,0.145264,0.262948,0.087274,0.289101,0.139072,0.320936,0.073392,0.322577,0.235353,0.159887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-16 03:37:37.341000+00:00,0.139889,0.336271,0.239999,0.215214,0.247496,0.400298,0.276191,0.215214,0.256104,0.087787,0.299147,0.112987,0.301299,0.107607,0.258256,0.168584,0.215214
2020-01-16 03:37:37.404000+00:00,0.140470,0.337669,0.240997,0.210705,0.248524,0.394397,0.277339,0.216108,0.257169,0.088152,0.300390,0.113457,0.302551,0.108054,0.259330,0.169285,0.216108
2020-01-16 03:37:37.427000+00:00,0.140243,0.337123,0.240607,0.210365,0.248123,0.393760,0.276891,0.215759,0.258911,0.088009,0.299905,0.122263,0.302062,0.107879,0.258911,0.169011,0.215759
2020-01-16 03:37:37.452000+00:00,0.139967,0.333767,0.240133,0.209950,0.247633,0.392983,0.276344,0.215333,0.258400,0.087836,0.299313,0.118433,0.312233,0.107667,0.258400,0.168678,0.215333


In [12]:
coor = model.predict(data.values).mean(axis=0)

In [13]:
np.expand_dims(coor, axis=0)

array([[23.49288,  8.3629 ]], dtype=float32)